In [1]:
import requests 
from bs4 import BeautifulSoup 
from datetime import datetime 
import pandas as pd 

# 뉴스 언론사 선택
news_company = ['중앙일보', '연합뉴스', '조선일보', '국민일보']


# USER-AGENT : your user-agent
# https://www.whatismybrowser.com/detect/what-is-my-user-agent에서 user-agent의 값 가져오기
headers = {"User-Agent": "USER-AGENT"}


#네이버 뉴스 랭킹 사이트
url = "https://news.naver.com/main/ranking/popularDay.naver"


# 서버 접속, html 해석
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text, "lxml")


# 언론사별 1 ~ 5위 데이터 담긴 div 모두 가져오기
all_box = soup.find_all("div", attrs = {"class":"rankingnews_box"})


# 각 언론사별 1 ~ 5위 데이터 접근, lst_all_data에 저장
lst_all_data = []
# count = 1 
for box in all_box:
    company_name = box.strong.text
    lst_all_rank = box.find_all("li")
    num_rank = 1
    for rank in lst_all_rank:
        lst_data = []
        lst_data.append(datetime.now().strftime("%Y-%m-%d"))
        lst_data.append(company_name)
        if rank.a == None:
                continue
        lst_data.append(num_rank)
        lst_data.append(rank.a.text)
        lst_data.append(rank.a["href"])
        num_rank += 1
        lst_all_data.append(lst_data)
    # 몇회째 실시 되고 있는지 확인
    # print(f"전체 {str(len(all_box))} 중 {count}회 종료")
    # count += 1


df = pd.DataFrame(lst_all_data, columns = ["date", "company", "rank", "title", "link"])


# 선택된 언론사의 뉴스만 남기고, index 정렬
df = df[ df['company'].isin(news_company)].reset_index()

In [2]:
import re
from tqdm import tqdm
import sys

final_urls = []
news_titles = []
news_contents = []
news_dates = []


for i in range(len(df)):
  final_urls.append(df['link'][i])

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    print(content)
    content = ''.join(str(content))
    print(content)
    if (i == 2):
      break

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    content = " ".join(re.split(r"\s+", content))
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern = pattern1, repl = '', string = str(news_date))

    news_dates.append(news_date)

  5%|████▏                                                                              | 1/20 [00:00<00:05,  3.50it/s]

[<div class="go_trans _article_content" id="dic_area">
			  주부 구모(44)씨는 지난 26일 배추를 절일 때 마당에 설치된 수도꼭지에서 고무호스를 빼냈다. 지난해 김장 때 역한 냄새 때문에 배추를 버린 경험 때문이었다.  <br/> <br/> <div class="ab_photo photo_center"> <div class="image"> <span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img class="_LAZY_LOADING" data-src="https://imgnews.pstatic.net/image/025/2022/11/27/0003241477_001_20221127174201104.jpg?type=w647" id="img1">
</img></div>
</div><em class="img_desc">지난 21일 전남 해남군 문내면 석교리의 한 배추밭에서 농민들이 배추를 수확하고 있다. 연합뉴스</em></span> <span class="mask"></span> </div> </div> <br/>   당시 친정집 마당에서 배추를 절였던 구씨는 수도꼭지에 끼워놓은 고무호스를 통해 물을 받았다. 하지만 몇 시간 뒤 절인 배추에서 역한 냄새가 나 결국 김장을 다시 해야 했다. 냄새 원인은 다름 아닌 수도꼭지에 달린 ‘고무호스’였다.  <br/> <br/> <div class="ab_sub_heading" style="position:relative;margin-top:17px;margin-bottom:16px;padding-top:15px;padding-bottom:14px;border-top:1px solid #444446;border-bottom:1px solid #ebebeb;color:#3e3e40;font-size:20

 10%|████████▎                                                                          | 2/20 [00:00<00:06,  2.68it/s]

[<div class="go_trans _article_content" id="dic_area">
<b style="font-size: inherit;">현명한 채식 실천법</b> <br/> <div class="ab_photo photo_center"> <div class="image"> <span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img class="_LAZY_LOADING" data-src="https://imgnews.pstatic.net/image/025/2022/11/27/0003241541_001_20221127195201103.jpg?type=w647" id="img1">
</img></div>
</div></span> <span class="mask"></span> </div> </div>        채식은 건강한 식사법의 하나로 여겨진다. 여기에 환경 보호와 동물권에 대한 관심이 더해지면서 자신을 드러내는 방식의 하나로    <br/>   채식을 실천하는 사람이 증가하고 있다. 하지만 건강 상태를 고려하지 않고 엄격히 채식만을 고집하는 것은 또 하나의 편식이라는 게 전문가들의 지적이다.    <br/>   고기 대신 선택하는 대체육 같은 제품은 맛을 내기 위해 여러 첨가물을 넣은 식물 가공식으로, 건강식과 거리가 멀다. 채식에 관한 오해와 현명한 채식 실천법을 알아본다.    <br/> <br/> <div class="ab_sub_heading" style="position:relative;margin-top:17px;margin-bottom:16px;padding-top:15px;padding-bottom:14px;border-top:1px solid #444446;border-bottom:1px solid #ebebeb;color:#3e3e40;font-s

 15%|████████████▍                                                                      | 3/20 [00:01<00:05,  2.97it/s]

...

[<div class="go_trans _article_content" id="dic_area">
<strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">종아리 다친 후 처음 훈련장에 나타났으나 따로 훈련…벤투 "내일 보고 결정"<br/>우루과이전 못 뛴 황희찬은 팀 훈련 소화 중이지만 가나전도 결장  </strong><br/><br/><span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img class="_LAZY_LOADING" data-src="https://imgnews.pstatic.net/image/001/2022/11/27/PYH2022112709500001300_P4_20221127200410577.jpg?type=w647" id="img1">
</img></div>
</div><em class="img_desc">자전거 타는 김민재<br/>(도하=연합뉴스) 김주성 기자 = 2022 카타르 월드컵 가나전을 하루 앞둔 27일 오후(현지시간) 부상 회복에 전념했던 김민재가 카타르 도하 알에글라 훈련장에서 팀 훈련에서 빠진 채 혼자 자전거를 타고 있다. 2022.11.27 utzza@yna.co.kr</em></span><br/><br/>    (도하=연합뉴스) 배진남 기자 = 종아리를 다친 축구대표팀 주축 중앙수비수 김민재(26·나폴리)가 가나와의 2022 카타르 월드컵 2차전에 출전할 수 있을까.<br/><br/>    김민재는 27일(한국시간) 축구대표팀 훈련이 진행된 카타르 도하 알에글라 훈련장에 동료들과 함께 모습을 드러냈다.<br/><br/>  

 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:06<00:00,  2.86it/s]

[<div class="go_trans _article_content" id="dic_area">
<span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img class="_LAZY_LOADING" data-src="https://imgnews.pstatic.net/image/001/2022/11/27/PYH2022112710930001300_P4_20221127194911894.jpg?type=w647" id="img1">
</img></div>
</div><em class="img_desc">카타르 월드컵 관중석에 등장한 욱일기<br/>(알라이얀=연합뉴스) 김도훈 기자 = 27일(현지시간) 카타르 알라이얀 아흐마드 빈 알리 스타디움에서 열린 2022 카타르 월드컵 조별리그 E조 일본과 코스타리카의 경기. 관중석에서 한 일본 축구 팬이 욱일기를 펼친 채 응원하고 있다. 2022.11.27 superdoo82@yna.co.kr</em></span><br/><br/>(알라이얀=연합뉴스) 장보인 기자 = 2022 국제축구연맹(FIFA) 카타르 월드컵 경기장에 욱일기가 등장했다.<br/><br/>    27일 일본과 코스타리카의 월드컵 조별리그 E조 2차전 경기가 열리는 카타르 알라이얀의 아흐마드 빈 알리 스타디움 관중석에선 욱일기가 펼쳐졌다.<br/><br/>    이날 경기장을 찾은 대다수의 일본 팬이 하얀 바탕에 붉은 원이 그려진 일본의 국기를 흔들었지만, 붉은 줄무늬가 그려진 욱일기도 군데군데서 포착됐다.<br/><br/>    경기장에 욱일기를 걸어 두려다 제지를 당하는 일본 팬도 있었다.<br/><br/>    욱일기는 일본이 19세기 말부터 태평양전쟁을 비롯한 아시아 침략 전쟁에 사용해 온 군대의 깃발로, 일본의 군국주의와 제국주의를 상징한다.<br/><br/>    유럽인들에게 

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  2.95it/s]

[<div class="go_trans _article_content" id="dic_area">
<strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">韓, 공동주거침입·보복범죄 혐의로 고발…경찰 출동</strong><br/><br/><span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img class="_LAZY_LOADING" data-src="https://imgnews.pstatic.net/image/001/2022/11/27/AKR20221127035051004_01_i_P4_20221127194609198.jpg?type=w647" id="img1">
</img></div>
</div><em class="img_desc">한동훈 장관 자택 찾아간 '더탐사' 취재진<br/>[유튜브 채널 '시민언론 더탐사' 캡처. 재판매 및 DB 금지]</em></span><br/><br/>    (서울=연합뉴스) 설하은 기자 = 유튜브 매체 '시민언론 더탐사'(이하 더탐사) 취재진이 27일 오후 한동훈 법무장관의 동의 또는 허락없이 그가 거주하는 아파트의 공동 현관을 통해 집 문 앞까지 찾아갔다.<br/><br/>    한 장관은 이에 이들을 공동주거침입 등 혐의로 고발했다.<br/><br/>    더탐사 유튜브 생중계 화면과 경찰에 따르면 더탐사 취재진 5명은 이날 오후 1시 30분께 한 장관이 거주하는 서울 강남구 도곡동의 한 주상복합 아파트를 찾았다.<br/><br/>    이들은 아파트 정문에서 "일요일에 경찰 수사관들이 갑자기 기습적으로 압수수색한 기자들의 마음

In [3]:
news_df = pd.DataFrame({'date': news_dates,'title': news_titles,'link': final_urls,'content': news_contents})

# #중복 행 지우기
# news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
# print("중복 제거 후 행 개수: ",len(news_df))

# #데이터 프레임 저장
# now = datetime.now() 
# news_df.to_excel('{}.xlsx'.format(now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

In [4]:
news_df

,date,title,link,content
0,2022-11-27 14:00:00,"""절인 배추서 역한 냄새, 다 버렸다""…김장 고무호스 주의보",https://n.news.naver.com/article/025/000324147...,[ 주부 구모(44)씨는 지난 26일 배추를 절일 때 마당에 설치된 수도꼭지에서 고...
1,2022-11-27 19:42:27,"몸에 좋은 채식인데…두부·시금치, 같이 먹으면 안되는 이유",https://n.news.naver.com/article/025/000324154...,[ 현명한 채식 실천법 채식은 건강한 식사법의 하나로 여겨진다. 여기에 환경 보호와...
2,2022-11-27 18:01:45,"황의조에 ""한국말 해라"" 그는 통역사 아니었다…통역 논란 진실",https://n.news.naver.com/article/025/000324152...,[ 2022 카타르 월드컵 대표팀 공격수 황의조가 14일(현지시간) 카타르 도하 알...
3,2022-11-27 16:57:01,"""압색 심정 느껴봐라""…한동훈 도곡동 집 도어락 누른 '더탐사'",https://n.news.naver.com/article/025/000324151...,[ 유튜브 '더탐사' 관계자 5명이 27일 한동훈 법무부 장관 아파트 현관 앞까지 ...
4,2022-11-27 19:35:48,"욱일기, 경기장에 걸었다 철거됐다…일본팬들 카타르 망신",https://n.news.naver.com/article/025/000324154...,[ 27일(현지시간) 카타르 알라이얀 아흐마드 빈 알리 스타디움에서 열린 2022 ...
5,2022-11-27 19:14:01,"더탐사, 한동훈 집 도어록 해제 시도... “압수수색 심정 느껴봐라”",https://n.news.naver.com/article/023/000373105...,[ 법조계 “도어록에 손대는 자체로 주거침입” 경찰서 찾아가선 “X소리” 욕설도 유...
6,2022-11-27 18:27:01,"尹, 술자리 의혹에 “동백아가씨 모르는 노래, 한동훈 2차 술자리 안가”",https://n.news.naver.com/article/023/000373105...,"[ 윤석열 대통령이 지난달 28일 오전 서울 용산 대통령실 청사로 출근, 야권발 '..."
7,2022-11-27 14:22:01,"“풀스윙, 스트레스 풀려”…강아지에 돌망치 17회 휘두른 동물카페 주인",https://n.news.naver.com/article/023/000373102...,[ 서울 마포구의 한 동물카페 사장이 매장 내 강아지에게 돌망치를 휘둘렀다. /SB...
8,2022-11-27 17:49:01,"발목 퉁퉁 부은 네이마르... 인스타에 “괜찮다, 가자!”",https://n.news.naver.com/article/023/000373104...,[ 오른쪽 발목 부상 29일 2차전 결장 브라질 공격수 네이마르(30·파리 생제르맹...
9,2022-11-27 17:57:01,“시진핑 타도” 구호 나왔다... 베이징·상하이서도 봉쇄 분노 시위,https://n.news.naver.com/article/023/000373104...,[ 27일(현지시간) 오전 중국 상하이에서 고강도 코로나 방역 정책에 반대하는 시민...
